In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
import random

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,Embedding,Bidirectional, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
from tqdm import tqdm

In [8]:
random.randint(1, 5)

1

In [9]:
0.0001 + (0.01+0.001)*random.randint(1, 5)

0.022099999999999998

In [20]:
np.random.uniform(0.0001, 0.01)

0.0034102445946021364

In [38]:
X = np.array([range(5),range(5)])
X

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [39]:
X[0][0] = 100
X

array([[100,   1,   2,   3,   4],
       [  0,   1,   2,   3,   4]])

In [45]:
x = np.zeros(50)
x = x.reshape(10, 5)
print(x)
x[0][0] = 100
print(x)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[100.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]


In [46]:
def initialize_populations(lower_bound, upper_bound, population, dimension): 
    bird_positions = np.zeros(population*dimension)
    bird_positions = bird_positions.reshape(population, dimension)
    for i in range(len(lower_bound)):
        lower = lower_bound[i]
        upper = upper_bound[i]
        col = []
        for j in range(population):
            pos = np.random.uniform(lower, upper)
            bird_positions[j, i] = pos
        
    
    return bird_positions

In [49]:
lower_bound = [0.0001, 1, 1, 1]
upper_bound = [0.01, 20, 20, 5]

population = 10
dimension = 4

pop = initialize_populations(lower_bound, upper_bound, population, dimension)
print(pop)

[[5.29849647e-03 3.42217392e+00 4.24222806e+00 2.08865374e+00]
 [8.44899149e-03 8.33550991e+00 1.00044734e+01 1.12355889e+00]
 [8.20791773e-03 2.25596938e+00 1.72199676e+01 2.58474028e+00]
 [7.60453324e-03 1.87136941e+01 1.36088408e+01 3.32586470e+00]
 [7.89017699e-03 6.90286297e+00 1.68527702e+01 4.83141563e+00]
 [9.48860906e-03 1.18993855e+01 3.33973379e+00 4.09943722e+00]
 [7.11474947e-03 1.67144144e+01 1.65874037e+00 1.88536840e+00]
 [4.76534821e-03 2.18789390e+00 1.24513022e+01 1.11045065e+00]
 [7.36524168e-03 1.13259194e+01 1.12339347e+01 2.56292151e+00]
 [3.69252529e-03 1.27418041e+01 2.79808056e+00 2.54673042e+00]]


In [99]:
def compare_mse(mse):
    bird_ranking = {}
    for i in range(len(mse)):
        bird_ranking[i] = mse[i]
        
    arranged_bird_ranking = list(dict(sorted(bird_ranking.items(), key=lambda x:x[1])).keys())
    return arranged_bird_ranking

In [100]:
mse = [1, 6, 9, 3, 2, 4]
ranking = compare_mse(mse)
print(ranking)

[0, 4, 3, 5, 1, 2]


In [101]:
def decide_updation(ranking):
    producers = ranking[0:2]
    monitors = ranking[2:4]
    scavengers = ranking[4:]
    
    return producers, monitors, scavengers

In [103]:
ranking = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(decide_updation(ranking))

([1, 2], [3, 4], [5, 6, 7, 8, 9, 10])


In [158]:
def update_producers(alpha, max_iter, Q, L, bird_index, dimension, alarm_value, safety_threshold, bird_matrix):
    new_bird_position = []
    for j in range(dimension):
        pos = bird_matrix[bird_index][j]
        L_pos = L[j]
        if alarm_value < safety_threshold:
            exp = np.exp((-i)/(alpha*max_iter))
            new_pos = pos*exp
        else:
            new_pos = pos + Q*L_pos
    
        new_bird_position.append(new_pos)
        
    return new_bird_position
                         

In [159]:
alpha = np.random.uniform()
max_iter = 5
Q = np.random.normal()
L = np.ones(5)
bird_index = 0
dimension = 5
alarm_value = 0.4 #np.random.uniform(0,1.1)
safety_threshold = 0.6 #np.random.uniform(0.5,1.1)
bird_matrix = population

new_pos = update_producers(alpha=alpha, max_iter=max_iter, Q=Q, L=L, bird_index=bird_index, dimension=dimension,
                           alarm_value=alarm_value, safety_threshold=safety_threshold, bird_matrix=bird_matrix)

print(new_pos)

[0.0002001367034876569, 0.36152421318312306, 4.229222396880705, 3.6293040976697717, 7.440668312994891]


In [160]:
alpha = np.random.uniform()
max_iter = 5
Q = np.random.normal()
L = np.ones(5)
bird_index = 0
dimension = 5
alarm_value = 0.7 #np.random.uniform(0,1.1)
safety_threshold = 0.6 #np.random.uniform(0.5,1.1)
bird_matrix = population

new_pos = update_producers(alpha=alpha, max_iter=max_iter, Q=Q, L=L, bird_index=bird_index, dimension=dimension,
                           alarm_value=alarm_value, safety_threshold=safety_threshold, bird_matrix=bird_matrix)

print(new_pos)

[0.3447398934469443, 4.065056275639939, 43.88820079006087, 37.71123661353281, 76.95434730498809]


In [150]:
#Chinese github = np.dot(np.abs(pos - optimal_pos), A_plus)*L

#Mealpy Github = np.abs(pos - optimal_pos)*np.random.uniform()

def update_scrounger(Q, worst_bird, bird_index, total_birds, best_bird, A_plus, L, bird_matrix, dimension):
    new_bird_position = []
    for j in range(dimension):
        pos = bird_matrix[bird_index][j]
        worst_bird_pos = worst_bird[j]
        optimal_pos = best_bird[j]
        if bird_index > (total_birds/2):
            exp = np.exp(((worst_bird_pos - pos)/bird_index**2))
            new_pos = Q*exp
        else:
            new_pos = optimal_pos + np.dot(np.dot(np.abs(pos - optimal_pos), A_plus),L)
            
        new_bird_position.append(new_pos)
        
    return new_bird_position

In [151]:
Q = np.random.normal()
worst_bird = population[8]
bird_index = 0
total_birds = 10
best_bird = population[2]
L = np.ones(5)
A_plus = 1/(np.transpose(L)*np.dot(L, np.transpose(L)))
bird_matrix = population
dimension = 5

new_pos = update_scrounger(Q=Q, worst_bird=worst_bird, bird_index=bird_index, total_birds=total_birds,
                           best_bird=best_bird, A_plus=A_plus, L=L, bird_matrix=bird_matrix, dimension=dimension)

print(new_pos)

[0.002060676011179666, 4.714397697666527, 43.545521572625105, 37.368557396097046, 88.73470773158375]


In [152]:
Q = np.random.normal()
worst_bird = population[8]
bird_index = 7
total_birds = 10
best_bird = population[2]
L = np.ones(5)
A_plus = 1/(np.transpose(L)*np.dot(L, np.transpose(L)))
bird_matrix = population
dimension = 5

new_pos = update_scrounger(Q=Q, worst_bird=worst_bird, bird_index=bird_index, total_birds=total_birds,
                           best_bird=best_bird, A_plus=A_plus, L=L, bird_matrix=bird_matrix, dimension=dimension)

print(new_pos)

[-1.6198494570875535, -1.6677462326529375, -2.012654702734999, -1.5220215397711028, -2.0124100655471815]


In [35]:
#the min the fitness val, better will be the positioning

def update_monitor(best_bird, beta, bird_index, K, worst_bird, fitness_matrix, epsillon, dimension, bird_matrix):
    new_bird_position = []
    current_bird = bird_matrix[bird_index]
    current_bird_fitness = fitness_matrix[bird_index][0]
    best_fitness = min(fitness_matrix)[0]
    worst_fitness = max(fitness_matrix)[0]
    for j in range(dimension):
        best_bird_pos = best_bird[j]
        worst_bird_pos = worst_bird[j]
        current_bird_pos = current_bird[j]       
        if current_bird_fitness > best_fitness:
            print("First condition")
            new_pos = best_bird_pos + beta*np.abs(current_bird_pos - best_bird_pos)  
        else:
            print("Other Condition")
            mul_val = np.abs(current_bird_pos - worst_bird_pos)/((current_bird_fitness-worst_fitness)+epsillon)
            new_pos = current_bird_pos + K*mul_val
        
        new_bird_position.append(new_pos)
    
    return new_bird_position

In [36]:
best_bird = population[0]
beta = np.random.uniform(-1, 1)
bird_index = 4
K = np.random.uniform(-1, 1)
worst_bird = population[2]
fitness_matrix = initial_mse
epsillon = 0.0000001
dimesion = 5
bird_matrix = population

new_pos = update_monitor(best_bird=best_bird, beta=beta, bird_index=bird_index, K=K, worst_bird=worst_bird, 
                         fitness_matrix=fitness_matrix, epsillon=epsillon, dimension=dimension, bird_matrix=bird_matrix)

print(new_pos)

First condition
First condition
First condition
First condition
First condition
[0.009805365143715447, 3.9802087598362377, 52.516029906356565, 39.32509991649552, 83.37393700387278]


In [37]:
best_bird = population[0]
beta = np.random.uniform(-1, 1)
bird_index = 8
K = np.random.uniform(-1, 1)
worst_bird = population[2]
fitness_matrix = initial_mse
epsillon = 0.0000001
dimesion = 5
bird_matrix = population

new_pos = update_monitor(best_bird=best_bird, beta=beta, bird_index=bird_index, K=K, worst_bird=worst_bird, 
                         fitness_matrix=fitness_matrix, epsillon=epsillon, dimension=dimension, bird_matrix=bird_matrix)

print(new_pos)

Other Condition
Other Condition
Other Condition
Other Condition
Other Condition
[0.008348216766626497, 5.855174794006461, 59.956752548201614, 94.68206735273557, 154.87166545781835]


In [83]:
b = np.array(list(map(lambda x: str(x).replace('0', '1'), np.random.randint(-1, 1, size=(5))))).astype('int64')
print(b)

[ 1 -1 -1 -1  1]


In [ ]:
def find_best_worst_bird(self, population, mse):
    worst_bird = mse.index(max(mse))
    best_bird = mse.index(min(mse))
    
    worst_bird_location = population[worst_bird]
    best_bird_location = population[best_bird]
    
    return worst_bird_location, best_bird_location

In [151]:
def initialize_populations(lower_bound, upper_bound, population, dimension): 
    bird_positions = np.zeros(population*dimension)
    bird_positions = bird_positions.reshape(population, dimension)
    for i in range(len(lower_bound)):
        lower = lower_bound[i]
        upper = upper_bound[i]
        col = []
        for j in range(population):
            pos = np.random.uniform(lower, upper)
            bird_positions[j, i] = pos
        
    
    return bird_positions

lower_bound = [0.0001, 1, 1, 1]
upper_bound = [0.01, 20, 20, 5]

population = 10
dimension = 4

pop = initialize_populations(lower_bound, upper_bound, population, dimension)
print(pop)

[[1.44462943e-03 3.87823697e+00 8.75998912e+00 2.26717653e+00]
 [9.64561694e-03 6.34561335e+00 1.40961549e+01 3.81759985e+00]
 [4.44533705e-03 1.50774171e+01 1.58522524e+01 2.51182355e+00]
 [1.45810388e-03 1.84785625e+01 2.18022049e+00 4.05904748e+00]
 [1.57454801e-03 3.07803036e+00 8.34470365e+00 3.34034316e+00]
 [9.66552960e-03 9.06037827e+00 6.24624874e+00 1.59044616e+00]
 [1.98970780e-03 8.11266354e+00 1.40493982e+01 2.11069477e+00]
 [6.94170774e-03 1.24713470e+01 9.70350651e+00 4.73838505e+00]
 [4.63714011e-03 1.36146292e+01 1.11848900e+01 3.37530815e+00]
 [6.99068576e-04 1.43932640e+00 4.63839036e+00 4.91848240e+00]]


In [152]:
new_pos = pop[0]
lower_bound = [0.0001, 1, 1, 1, 1]
upper_bound = [0.01, 20, 20, 5, 50]

for i in range(len(new_pos)):
    
    condition = np.logical_and(lower_bound[i] <= new_pos[i], new_pos[i] <= upper_bound[i])
    pos_rand = np.random.uniform(lower_bound[i], upper_bound[i])
    print(condition)
    print(np.where(condition, new_pos[i], pos_rand))

True
0.0014446294345966055
True
3.878236968875145
True
8.759989115828681
True
2.2671765282729965


# Connecting SSA and Model

In [206]:
class SSA_new():
    def __init__(self, total_birds, dimension, maximum_iterations):
        print("Inititializing parameters")
        #basic paramters 
        self.dimension = dimension
        self.total_birds = total_birds
        #update producer
        self.alpha = np.random.uniform(0, 1)
        self.T = maximum_iterations
        self.Q = np.random.normal()
        self.L = np.ones(dimension)
        self.alarm_value = np.random.uniform(0, 1)
        self.safety_threshold = np.random.uniform(0.5, 1)
        #update scrounger
        self.A = np.array(list(map(lambda x: str(x).replace('0', '1'), np.random.randint(-1, 1, size=(dimension))))).astype('int64')
        #update monitor
        self.beta = np.random.uniform(-1, 1)
        self.K = np.random.uniform(-1, 1)
        self.epsillon = 0.0000001
        
    def initialize_populations(self, lower_bound, upper_bound): 
        bird_positions = np.zeros(self.total_birds*self.dimension)
        bird_positions = bird_positions.reshape(self.total_birds, self.dimension)
        for i in range(len(lower_bound)):
            lower = lower_bound[i]
            upper = upper_bound[i]
            col = []
            for j in range(self.total_birds):
                pos = np.random.uniform(lower, upper)
                bird_positions[j, i] = pos
        
    
        return bird_positions
    
    def compare_mse_and_decide_updation(self, mse):
        bird_ranking = {}
        for i in range(len(mse)):
            bird_ranking[i] = mse[i]
        
        arranged_bird_ranking = list(dict(sorted(bird_ranking.items(), key=lambda x:x[1])).keys())
        producers = arranged_bird_ranking[0:2]
        monitors = arranged_bird_ranking[2:4]
        scroungers = arranged_bird_ranking[4:]
    
        return producers, monitors, scroungers
    
    def check_limits(self, new_pos, lower_bound, upper_bound):
        condition = np.logical_and(lower_bound <= new_pos, new_pos <= upper_bound)
        pos_rand = np.random.uniform(lower_bound, upper_bound)
        return np.where(condition, new_pos, pos_rand)
    
    def find_best_worst_bird(self, population, mse):
        worst_bird = mse.index(max(mse))
        best_bird = mse.index(min(mse))
    
        worst_bird_location = population[worst_bird]
        best_bird_location = population[best_bird]
    
        return worst_bird_location, best_bird_location
    
    def update_producers(self, bird_index, bird_matrix):
        new_bird_position = []
        for j in range(self.dimension):
            pos = bird_matrix[bird_index][j]
            L_pos = self.L[j]
            if self.alarm_value < self.safety_threshold:
                exp = np.exp((-i)/(self.alpha*self.T))
                new_pos = pos*exp
            else:
                new_pos = pos + self.Q*L_pos
    
            new_bird_position.append(new_pos)
        
        return np.array(new_bird_position)
    
    
    def update_scrounger(self, worst_bird, bird_index, best_bird, bird_matrix):
        new_bird_position = []
        for j in range(self.dimension):
            pos = bird_matrix[bird_index][j]
            worst_bird_pos = worst_bird[j]
            optimal_pos = best_bird[j]
            if bird_index > (self.total_birds/2):
                exp = np.exp(((worst_bird_pos - pos)/bird_index**2))
                new_pos = self.Q*exp
            else:
                A_plus = 1/(np.transpose(self.A)*np.dot(self.L, np.transpose(self.A)))
                new_pos = optimal_pos + np.dot(np.dot(np.abs(pos - optimal_pos), A_plus),self.L)
            
            new_bird_position.append(new_pos)
        
        return np.array(new_bird_position)
    
    def update_monitor(self, best_bird, bird_index, worst_bird, fitness_matrix, bird_matrix):
        new_bird_position = []
        current_bird = bird_matrix[bird_index]
        current_bird_fitness = fitness_matrix[bird_index][0]
        best_fitness = min(fitness_matrix)[0]
        worst_fitness = max(fitness_matrix)[0]
        for j in range(self.dimension):
            best_bird_pos = best_bird[j]
            worst_bird_pos = worst_bird[j]
            current_bird_pos = current_bird[j]       
            if current_bird_fitness > best_fitness:
                new_pos = best_bird_pos + self.beta*np.abs(current_bird_pos - best_bird_pos)  
            else:
                mul_val = np.abs(current_bird_pos - worst_bird_pos)/((current_bird_fitness-worst_fitness)+self.epsillon)
                new_pos = current_bird_pos + self.K*mul_val
        
            new_bird_position.append(new_pos)
    
        return np.array(new_bird_position)
        

In [5]:
class SSA():
    def __init__(self, n_dim, max_iter, lb, ub, pop_size):
        self.n_dim = n_dim  #hyperparameters
        self.pop = pop_size #no. of sparrow
        self.producers = (20*pop_size)/100 #20% of population 
        self.monitors =  (20*pop_size)/100 #20% of population
        self.scrounger = pop_size - self.producers - self.monitors
        self.safety_threshold = 0.8
        
    def initialize_populations(self, lower_bound, upper_bound, population, dimension): 
        bird_positions = np.zeros(population*dimension)
        bird_positions = bird_positions.reshape(population, dimension)
        for i in range(len(lower_bound)):
            lower = lower_bound[i]
            upper = upper_bound[i]
            col = []
            for j in range(population):
                pos = np.random.uniform(lower, upper)
                bird_positions[j, i] = pos


        return bird_positions
        

In [198]:
class Model():
    def __init__(self, learning_rate, epochs, neuron_1, neuron_2, neuron_3):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.neuron_1 = neuron_1
        self.neuron_2 = neuron_2
        self.neuron_3 = neuron_3
        
    def build_model(self, vocab_size, embedding_matrix):
        model_2 = Sequential()
        model_2.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 1100, trainable = False))

        model_2.add(Bidirectional(LSTM(self.neuron_1, return_sequences = True)))
        model_2.add(Bidirectional(LSTM(self.neuron_2, return_sequences = True)))
        model_2.add(Conv1D(self.neuron_3, 3, activation='tanh', padding = 'same'))
        model_2.add(Flatten())

        model_2.add(Dropout(0.2))

        model_2.add(Dense(256,activation = 'tanh'))
        model_2.add(Dense(256,activation = 'tanh'))
        model_2.add(Dense(128,activation = 'tanh'))


        model_2.add(Dense(1,activation = 'sigmoid'))
        opt = Adam(learning_rate=self.learning_rate)
        model_2.compile(loss='binary_crossentropy', optimizer=opt, metrics = ['accuracy'])
        
        return model_2
    
    def train_test_model(self, model, x_train, y_train, x_test, y_test):
        history = model.fit(x_train, y_train, batch_size = 128, verbose = 1, epochs = self.epochs, validation_split = 0.2)
        test_output = model.predict(x_test)
        diff_square = []

        for i in range(len(y_test)):
            val = (y_test[i] - test_output[i][0])**2
            diff_square.append(val)
    
        mse = sum(diff_square)/len(y_test)
        return mse
        

In [199]:
class Dataset():
    def generate_dataset_train(self):
        df = pd.read_csv("sentiment_dataset_merged.csv")
        df = df.drop(["Unnamed: 0"], axis = 1)
        x_vocab = df['Review']
        embedding_vector = {}
        f = open('glove.6B.200d.txt')
        for line in tqdm(f):
            value = line.split(' ')
            word = value[0]
            coef = np.array(value[1:],dtype = 'float32')
            embedding_vector[word] = coef
            
        token = Tokenizer()
        token.fit_on_texts(x_vocab)
        
        df_train = pd.read_csv("train_combined.csv")
        df_train = df_train.drop(['Unnamed: 0'], axis = 1)
        
        x_train = list(df_train['Review'])
        y_train = df_train['Sentiment']
        
        x_filtered = []
        y_filtered = []
        for i in range(0, len(x_train)-1):
            words = len(x_train[i].split())
            if len(x_train[i].split()) < 1100:
                x_filtered.append(x_train[i])
                y_filtered.append(y_train[i])
                
        #vocab size
        vocab_size = len(token.word_index)+1
        
        #x_train
        seq = token.texts_to_sequences(x_filtered)
        pad_seq = pad_sequences(seq, maxlen=1100, padding = 'post')
        
        #embedding_matrix
        embedding_matrix = np.zeros((vocab_size,200))
        for word,i in tqdm(token.word_index.items()):
            embedding_value = embedding_vector.get(word)
            if embedding_value is not None:
                embedding_matrix[i] = embedding_value
         
        sentiment = {
            "Negative" : 0,
            "Positive" : 1
        }

        y_filtered_converted = []
        for sent in y_filtered:
            y_filtered_converted.append(sentiment[sent])
         
        #y_train
        y_filtered_converted = np.int64(y_filtered_converted)
        
        return vocab_size, pad_seq, embedding_matrix, y_filtered_converted, token
    
    def generate_dataset_test(self, token):
        df_test = pd.read_csv("test_combined.csv")
        df_test = df_test.drop(['Unnamed: 0'], axis = 1)
        
        x_test = df_test['Review']
        y_test = list(df_test['Sentiment'])

        x_test_filtered = []
        y_test_filtered = []
        for i in range(0, len(x_test)-1):
            words = len(x_test[i].split())
            if len(x_test[i].split()) < 1100:
                x_test_filtered.append(x_test[i])
                y_test_filtered.append(y_test[i])
            
        #x_test    
        seq_test = token.texts_to_sequences(x_test_filtered)
        pad_seq_test = pad_sequences(seq_test, maxlen=1100, padding = 'post')
        
        sentiment = {
            "Negative" : 0,
            "Positive" : 1
        }

        y_test_filtered_converted = []
        for sent in y_test_filtered:
            y_test_filtered_converted.append(sentiment[sent])
            
        #y_test    
        y_test_filtered_converted = np.int64(y_test_filtered_converted)
        
        return pad_seq_test, y_test_filtered_converted

# Call the classes 

In [200]:
dataset = Dataset()
vocab_size, x_train, embedding_matrix, y_train, token = dataset.generate_dataset_train()
x_text, y_test = dataset.generate_dataset_test(token)

400000it [00:07, 52210.28it/s]
100%|███████████████████████████████| 120904/120904 [00:00<00:00, 582355.44it/s]


In [9]:
lower_bound = [0.0001, 1, 20, 20, 50]
upper_bound = [0.01, 5, 50, 50, 100]

population = 10
dimension = 5


ssa = SSA(dimension, 10, lower_bound, upper_bound, population)
population = ssa.initialize_populations(lower_bound, upper_bound, population, dimension)

In [10]:
print(population)

[[7.42941888e-03 2.46066488e+00 4.27421773e+01 3.44223157e+01
  7.24400390e+01]
 [8.22917241e-04 3.16766235e+00 4.48662256e+01 4.60603448e+01
  5.55948508e+01]
 [6.07902114e-03 2.96623780e+00 4.61375746e+01 4.42545946e+01
  9.28096747e+01]
 [4.76027625e-03 1.64687250e+00 2.02482933e+01 2.76676339e+01
  9.09529936e+01]
 [4.09734959e-03 4.59170024e+00 2.90351525e+01 4.12980675e+01
  8.77739320e+01]
 [8.37029744e-03 4.19531256e+00 4.80896913e+01 2.13911798e+01
  7.37367370e+01]
 [5.83755805e-03 4.60411252e+00 4.37853503e+01 4.86214919e+01
  5.58063908e+01]
 [9.12668758e-03 4.85323020e+00 4.34500781e+01 2.43859888e+01
  5.89234426e+01]
 [5.22359684e-03 1.87718805e+00 4.91077076e+01 2.52448251e+01
  6.94140122e+01]
 [1.51054750e-03 3.38915146e+00 3.14304347e+01 3.86069332e+01
  9.95348976e+01]]


In [11]:
population.shape[0]

10

In [12]:
initial_mse = []
for i in range(population.shape[0]):
    learning_rate = population[i][0]
    epochs = math.floor(population[i][1])
    neuron_1 = math.floor(population[i][2])
    neuron_2 = math.floor(population[i][3])
    neuron_3 = math.floor(population[i][4])
    
    model = Model(learning_rate=learning_rate, epochs=epochs, neuron_1=neuron_1, neuron_2=neuron_2, neuron_3=neuron_3)
    trial_model = model.build_model(vocab_size=vocab_size, embedding_matrix=embedding_matrix)
    mse = model.train_test_model(model=trial_model, x_train=x_train, x_test=x_text, y_test=y_test, y_train=y_train)
    
    print(mse)
    initial_mse.append(mse)

2023-01-29 19:46:43.439561: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2
157/157 [==============================] - 267s 2s/step - loss: 0.7586 - accuracy: 0.4993 - val_loss: 0.7011 - val_accuracy: 0.4946
Epoch 2/2
157/157 [==============================] - 269s 2s/step - loss: 0.6983 - accuracy: 0.4987 - val_loss: 0.7178 - val_accuracy: 0.4946
[0.3655005]
Epoch 1/3
157/157 [==============================] - 310s 2s/step - loss: 0.5444 - accuracy: 0.7349 - val_loss: 0.5169 - val_accuracy: 0.7644
Epoch 2/3
157/157 [==============================] - 303s 2s/step - loss: 0.3832 - accuracy: 0.8304 - val_loss: 0.3931 - val_accuracy: 0.8259
Epoch 3/3
157/157 [==============================] - 297s 2s/step - loss: 0.3437 - accuracy: 0.8478 - val_loss: 0.3658 - val_accuracy: 0.8337
[0.29538572]
Epoch 1/2
157/157 [==============================] - 314s 2s/step - loss: 0.7378 - accuracy: 0.5065 - val_loss: 0.6984 - val_accuracy: 0.5054
Epoch 2/2
157/157 [==============================] - 316s 2s/step - loss: 0.6997 - accuracy: 0.5006 - val_loss: 0.6947 - va

In [13]:
initial_mse

[array([0.3655005], dtype=float32),
 array([0.29538572], dtype=float32),
 array([0.21750739], dtype=float32),
 array([0.26299053], dtype=float32),
 array([0.2737737], dtype=float32),
 array([0.32697344], dtype=float32),
 array([0.3092671], dtype=float32),
 array([0.18779376], dtype=float32),
 array([0.17117453], dtype=float32),
 array([0.4166136], dtype=float32)]

In [30]:
fitness_matrix.index(min(fitness_matrix))

8

In [32]:
min(fitness_matrix)[0]

0.17117453

# Call the SSA_new

In [160]:
dimension = 5
total_birds = 10
max_iterations = 5
ssa = SSA_new(dimension=dimension, total_birds=total_birds, maximum_iterations=max_iterations)

Inititializing parameters


In [132]:
lower_bound = [0.0001, 1, 20, 20, 50]
upper_bound = [0.01, 5, 50, 50, 100]

initial_population = ssa.initialize_populations(lower_bound=lower_bound, upper_bound=upper_bound)
print(initial_population)

[[4.41509280e-03 3.57111471e+00 4.40854028e+01 3.76772393e+01
  5.04367761e+01]
 [2.48232594e-03 2.65857229e+00 4.12531055e+01 3.58287820e+01
  6.32694913e+01]
 [2.57356197e-03 1.90114360e+00 3.67106759e+01 3.55062613e+01
  9.14105918e+01]
 [1.44831836e-03 1.11407302e+00 3.46970481e+01 3.04967156e+01
  8.36392515e+01]
 [2.43950647e-03 1.30408590e+00 4.27986658e+01 4.11964106e+01
  6.88471699e+01]
 [7.57312472e-03 3.37862116e+00 2.18215192e+01 4.41805341e+01
  8.13236330e+01]
 [8.97741753e-03 3.76267653e+00 4.60044105e+01 4.45034140e+01
  6.33263080e+01]
 [8.71076019e-03 1.13891901e+00 4.50635956e+01 3.11048052e+01
  9.90332664e+01]
 [6.56899217e-03 1.92356612e+00 3.73717979e+01 2.81796852e+01
  5.63258584e+01]
 [2.50457615e-03 3.01718137e+00 2.67555986e+01 3.45339193e+01
  8.74847307e+01]]


In [133]:
initial_mse

[array([0.3655005], dtype=float32),
 array([0.29538572], dtype=float32),
 array([0.21750739], dtype=float32),
 array([0.26299053], dtype=float32),
 array([0.2737737], dtype=float32),
 array([0.32697344], dtype=float32),
 array([0.3092671], dtype=float32),
 array([0.18779376], dtype=float32),
 array([0.17117453], dtype=float32),
 array([0.4166136], dtype=float32)]

In [134]:
producers, monitors, scroungers = ssa.compare_mse_and_decide_updation(initial_mse)
print(producers)
print(monitors)
print(scroungers)

[8, 7]
[2, 3]
[4, 1, 6, 5, 0, 9]


In [135]:
for producer in producers:
    new_location = ssa.update_producers(bird_index=producer, bird_matrix=population)
    print(new_location)

[3.44651834e-04 1.23856478e-01 3.24011634e+00 1.66564831e+00
 4.57992210e+00]
[6.02176951e-04 3.20215120e-01 2.86682712e+00 1.60898247e+00
 3.88775650e+00]


In [136]:
best_bird, worst_bird = ssa.find_best_worst_bird(population=population, mse=initial_mse)
print(best_bird)
print(worst_bird)

[1.51054750e-03 3.38915146e+00 3.14304347e+01 3.86069332e+01
 9.95348976e+01]
[5.22359684e-03 1.87718805e+00 4.91077076e+01 2.52448251e+01
 6.94140122e+01]


In [137]:
for monitor in monitors:
    new_location = ssa.update_monitor(bird_index=monitor, bird_matrix=population, 
                                      worst_bird=worst_bird, fitness_matrix=initial_mse, best_bird=best_bird)
    
    print(new_location)

[2.86014094e-03 3.51408630e+00 3.57751373e+01 4.02753344e+01
 1.01521626e+02]
[2.47056455e-03 3.90384596e+00 3.47338017e+01 4.18385611e+01
 1.02070117e+02]


In [138]:
for scrounger in scroungers:
    new_location = ssa.update_scrounger(worst_bird=worst_bird, bird_index=scrounger, best_bird=best_bird, 
                                        bird_matrix=population)
    
    print(new_location)

[4.09734959e-03 4.59170024e+00 3.38257170e+01 4.12980675e+01
 1.11295863e+02]
[2.19817776e-03 3.61064057e+00 4.48662256e+01 4.60603448e+01
 1.43474944e+02]
[1.31406414 1.21822379 1.5234615  0.68645812 1.91770905]
[8.37029744e-03 4.19531256e+00 4.80896913e+01 5.58226866e+01
 1.25333058e+02]
[7.42941888e-03 4.31763804e+00 4.27421773e+01 4.27915508e+01
 1.26629756e+02]
[1.31414679 1.28978505 1.63456992 1.1142455  0.90599545]


In [171]:
def get_new_population(ssa, population, mse, lower_bound, upper_bound):
    producers, monitors, scroungers = ssa.compare_mse_and_decide_updation(mse)
    best_bird, worst_bird = ssa.find_best_worst_bird(population=population, mse=mse)
    
    for producer in producers:
        new_location = ssa.update_producers(bird_index=producer, bird_matrix=population)
        for i in range(len(new_location)):
            loc = ssa.check_limits(lower_bound=lower_bound[i], upper_bound=upper_bound[i], new_pos=new_location[i])
            if loc != new_location[i]:
                print("Value Changed at index " + str(i))
                new_location[i] = loc
            else:
                continue
        population[producer] = new_location
        
    for monitor in monitors:
        new_location = ssa.update_monitor(bird_index=monitor, bird_matrix=population, 
                                      worst_bird=worst_bird, fitness_matrix=initial_mse, best_bird=best_bird)
        for i in range(len(new_location)):
            loc = ssa.check_limits(lower_bound=lower_bound[i], upper_bound=upper_bound[i], new_pos=new_location[i])
            if loc != new_location[i]:
                print("Value Changed at index " + str(i))
                new_location[i] = loc
            else:
                continue
        population[monitor] = new_location
        
    for scrounger in scroungers:
        new_location = ssa.update_scrounger(worst_bird=worst_bird, bird_index=scrounger, best_bird=best_bird, 
                                        bird_matrix=population)
        for i in range(len(new_location)):
            loc = ssa.check_limits(lower_bound=lower_bound[i], upper_bound=upper_bound[i], new_pos=new_location[i])
            if loc != new_location[i]:
                print("Value Changed at index " + str(i))
                new_location[i] = loc
            else:
                continue
        population[scrounger] = new_location
        
    return population

In [161]:
lower_bound = [0.0001, 1, 20, 20, 50]
upper_bound = [0.01, 5, 50, 50, 100]
pop = get_new_population(ssa, initial_population, initial_mse, lower_bound, upper_bound)
print(pop)

Value Changed at index 3
Value Changed at index 4
Value Changed at index 2
Value Changed at index 3
Value Changed at index 4
Value Changed at index 1
Value Changed at index 4
Value Changed at index 4
Value Changed at index 0
Value Changed at index 2
Value Changed at index 3
Value Changed at index 4
Value Changed at index 1
Value Changed at index 4
Value Changed at index 0
Value Changed at index 2
Value Changed at index 3
Value Changed at index 4
[[7.97008279e-03 3.89673322e+00 3.14053391e+01 4.04465464e+01
  9.96792966e+01]
 [7.73175214e-03 3.67089905e+00 3.32148137e+01 4.14972977e+01
  9.56815013e+01]
 [3.39501751e-03 4.69398862e+00 2.79711986e+01 3.96181416e+01
  9.53831631e+01]
 [3.06923619e-03 3.13708748e+00 3.20448192e+01 4.75920476e+01
  8.67343110e+01]
 [5.13080579e-03 4.31818105e+00 3.35057443e+01 4.76685736e+01
  5.88563890e+01]
 [5.45172156e-03 2.50955976e+00 3.96706304e+01 4.92488299e+01
  8.88577403e+01]
 [3.45188154e-03 1.22386808e+00 4.61070746e+01 4.45657329e+01
  7.4224

# Final Implementation

In [166]:
def train_model(population):
    mse_vals = []
    for i in range(population.shape[0]):
        learning_rate = population[i][0]
        epochs = math.floor(population[i][1])
        neuron_1 = math.floor(population[i][2])
        neuron_2 = math.floor(population[i][3])
        neuron_3 = math.floor(population[i][4])
    
        model = Model(learning_rate=learning_rate, epochs=epochs, neuron_1=neuron_1, neuron_2=neuron_2, 
                      neuron_3=neuron_3)
        trial_model = model.build_model(vocab_size=vocab_size, embedding_matrix=embedding_matrix)
        mse = model.train_test_model(model=trial_model, x_train=x_train, x_test=x_text, y_test=y_test, 
                                     y_train=y_train)
    
        mse_vals.append(mse)
        
    return mse_vals

In [174]:
dimension = 5
total_birds = 10
max_iterations = 5
ssa = SSA_new(dimension=dimension, total_birds=total_birds, maximum_iterations=max_iterations)

lower_bound = [0.0001, 1, 20, 20, 50]
upper_bound = [0.01, 5, 50, 50, 100]

population = ssa.initialize_populations(lower_bound=lower_bound, upper_bound=upper_bound)

mse_vals_trial = []
population_trials = []

for i in range(max_iterations):
    mse_vals = train_model(population)
    print("===================== Iteration " + str(i) + " =========================")
    print(mse_vals)
    mse_vals_trial.append(mse_vals)
    population = get_new_population(ssa, population, mse_vals, lower_bound=lower_bound, upper_bound=upper_bound)
    print("===================== New Population " + " =========================")
    print(population)
    population_trials.append(population)

Inititializing parameters
157/157 [==============================] - 262s 2s/step - loss: 0.7279 - accuracy: 0.5018 - val_loss: 0.6937 - val_accuracy: 0.4946
Epoch 1/2
157/157 [==============================] - 167s 1s/step - loss: 0.7271 - accuracy: 0.4995 - val_loss: 0.6966 - val_accuracy: 0.4946
Epoch 2/2
157/157 [==============================] - 261s 2s/step - loss: 0.7398 - accuracy: 0.4983 - val_loss: 0.6958 - val_accuracy: 0.4946
Epoch 1/2
157/157 [==============================] - 246s 2s/step - loss: 0.7084 - accuracy: 0.5027 - val_loss: 0.6934 - val_accuracy: 0.5062
Epoch 2/2
157/157 [==============================] - 246s 2s/step - loss: 0.6123 - accuracy: 0.6402 - val_loss: 0.4918 - val_accuracy: 0.7782
Epoch 1/4
157/157 [==============================] - 279s 2s/step - loss: 0.7387 - accuracy: 0.4974 - val_loss: 0.6938 - val_accuracy: 0.5054
Epoch 2/4
157/157 [==============================] - 278s 2s/step - loss: 0.6981 - accuracy: 0.4981 - val_loss: 0.7028 - val_accurac

157/157 [==============================] - 169s 1s/step - loss: 0.5590 - accuracy: 0.7032 - val_loss: 0.4847 - val_accuracy: 0.7712
Epoch 1/2
157/157 [==============================] - 201s 1s/step - loss: 0.7382 - accuracy: 0.5008 - val_loss: 0.7109 - val_accuracy: 0.5054
Epoch 2/2
157/157 [==============================] - 194s 1s/step - loss: 0.6978 - accuracy: 0.5014 - val_loss: 0.7099 - val_accuracy: 0.5054
Epoch 1/2
157/157 [==============================] - 212s 1s/step - loss: 0.7541 - accuracy: 0.5005 - val_loss: 0.6942 - val_accuracy: 0.4946
Epoch 2/2
157/157 [==============================] - 198s 1s/step - loss: 0.6980 - accuracy: 0.4990 - val_loss: 0.7051 - val_accuracy: 0.4946
Epoch 1/2
157/157 [==============================] - 247s 2s/step - loss: 0.5656 - accuracy: 0.7190 - val_loss: 0.4174 - val_accuracy: 0.8183
Epoch 2/2
157/157 [==============================] - 245s 2s/step - loss: 0.3880 - accuracy: 0.8281 - val_loss: 0.3850 - val_accuracy: 0.8389
Epoch 1/4
157/15

Epoch 1/4
157/157 [==============================] - 266s 2s/step - loss: 0.7286 - accuracy: 0.4965 - val_loss: 0.6930 - val_accuracy: 0.5054
Epoch 2/4
157/157 [==============================] - 270s 2s/step - loss: 0.6951 - accuracy: 0.5028 - val_loss: 0.6931 - val_accuracy: 0.5192
Epoch 3/4
157/157 [==============================] - 265s 2s/step - loss: 0.6954 - accuracy: 0.4964 - val_loss: 0.6933 - val_accuracy: 0.5054
Epoch 4/4
157/157 [==============================] - 265s 2s/step - loss: 0.6951 - accuracy: 0.5011 - val_loss: 0.6929 - val_accuracy: 0.5819
Epoch 1/4
157/157 [==============================] - 329s 2s/step - loss: 0.7357 - accuracy: 0.5031 - val_loss: 0.6955 - val_accuracy: 0.4946
Epoch 2/4
157/157 [==============================] - 317s 2s/step - loss: 0.6956 - accuracy: 0.5006 - val_loss: 0.6928 - val_accuracy: 0.5184
Epoch 3/4
157/157 [==============================] - 317s 2s/step - loss: 0.6949 - accuracy: 0.5053 - val_loss: 0.6970 - val_accuracy: 0.5054
Epoch 

In [175]:
mse_vals_trial

[[array([0.26383072], dtype=float32),
  array([0.21036917], dtype=float32),
  array([0.28235334], dtype=float32),
  array([0.5008182], dtype=float32),
  array([0.3229772], dtype=float32),
  array([0.3977326], dtype=float32),
  array([0.41356224], dtype=float32),
  array([0.24496287], dtype=float32),
  array([0.4587565], dtype=float32),
  array([0.2937598], dtype=float32)],
 [array([0.42803812], dtype=float32),
  array([0.2748334], dtype=float32),
  array([0.35298187], dtype=float32),
  array([0.42002594], dtype=float32),
  array([0.2767299], dtype=float32),
  array([0.18704458], dtype=float32),
  array([0.18397796], dtype=float32),
  array([0.2677324], dtype=float32),
  array([0.26998937], dtype=float32),
  array([0.41944915], dtype=float32)],
 [array([0.5146933], dtype=float32),
  array([0.1630953], dtype=float32),
  array([0.32690898], dtype=float32),
  array([0.50750333], dtype=float32),
  array([0.2207385], dtype=float32),
  array([0.2975065], dtype=float32),
  array([0.2795158], d

# Reflection method to update values

In [189]:
def reflect(position, lower_bound, upper_bound):
    new_position = position
    for i in range(len(position)):
        if position[i] < lower_bound[i]:
            new_position[i] = lower_bound[i] + (lower_bound[i] - position[i])
        elif position[i] > upper_bound[i]:
            new_position[i] = upper_bound[i] - (position[i] - upper_bound[i])
    return new_position

In [190]:
def get_new_population_reflect(ssa, population, mse, lower_bound, upper_bound):
    producers, monitors, scroungers = ssa.compare_mse_and_decide_updation(mse)
    best_bird, worst_bird = ssa.find_best_worst_bird(population=population, mse=mse)
    
    for producer in producers:
        new_location = ssa.update_producers(bird_index=producer, bird_matrix=population)
        new_location = reflect(position=new_location, lower_bound=lower_bound, upper_bound=upper_bound)
        population[producer] = new_location
        
    for monitor in monitors:
        new_location = ssa.update_monitor(bird_index=monitor, bird_matrix=population, 
                                      worst_bird=worst_bird, fitness_matrix=initial_mse, best_bird=best_bird)
        new_location = reflect(position=new_location, lower_bound=lower_bound, upper_bound=upper_bound)
        population[monitor] = new_location
        
    for scrounger in scroungers:
        new_location = ssa.update_scrounger(worst_bird=worst_bird, bird_index=scrounger, best_bird=best_bird, 
                                        bird_matrix=population)
        new_location = reflect(position=new_location, lower_bound=lower_bound, upper_bound=upper_bound)
        population[scrounger] = new_location
        
    return population

In [192]:
lower_bound = [0.0001, 1, 20, 20, 50]
upper_bound = [0.01, 5, 50, 50, 100]
pop = get_new_population_reflect(ssa, initial_population, initial_mse, lower_bound, upper_bound)
print(pop)

[[8.04852941e-03 4.18248130e+00 4.21533768e+01 3.72019720e+01
  9.12718396e+01]
 [7.23377413e-03 3.29219477e+00 4.69345554e+01 3.93850540e+01
  9.61983588e+01]
 [1.62545780e-03 1.37147678e+00 3.81619348e+01 2.48218545e+01
  5.72001095e+01]
 [3.70602160e-03 2.55111267e+00 3.54373722e+01 2.48256627e+01
  5.89303630e+01]
 [7.25697718e-03 3.83016380e+00 4.11922817e+01 4.28187548e+01
  8.94340214e+01]
 [7.33404171e-03 2.93484549e+00 4.37376743e+01 4.08187956e+01
  9.79360818e+01]
 [5.17543266e-01 2.49394271e+00 4.04761177e+01 4.08011423e+01
  1.00737405e+02]
 [1.33538044e-04 1.96739599e+00 3.91626785e+01 3.91722147e+01
  9.87865605e+01]
 [1.38875039e-04 1.97348557e+00 3.93189888e+01 3.92586074e+01
  9.86569725e+01]
 [5.17503637e-01 2.51349299e+00 4.05117983e+01 4.05938795e+01
  1.00891781e+02]]


In [193]:
dimension = 5
total_birds = 5
max_iterations = 3
ssa = SSA_new(dimension=dimension, total_birds=total_birds, maximum_iterations=max_iterations)

lower_bound = [0.0001, 1, 20, 20, 50]
upper_bound = [0.01, 5, 50, 50, 100]

population = ssa.initialize_populations(lower_bound=lower_bound, upper_bound=upper_bound)

mse_vals_trial = []
population_trials = []

for i in range(max_iterations):
    mse_vals = train_model(population)
    print("===================== Iteration " + str(i) + " =========================")
    print(mse_vals)
    mse_vals_trial.append(mse_vals)
    population = get_new_population_reflect(ssa, population, mse_vals, lower_bound=lower_bound, upper_bound=upper_bound)
    print("===================== New Population " + " =========================")
    print(population)
    population_trials.append(population)

Inititializing parameters
Epoch 1/3
157/157 [==============================] - 253s 2s/step - loss: 0.7402 - accuracy: 0.4984 - val_loss: 0.6991 - val_accuracy: 0.5074
Epoch 2/3
157/157 [==============================] - 254s 2s/step - loss: 0.6967 - accuracy: 0.5043 - val_loss: 0.6988 - val_accuracy: 0.4928
Epoch 3/3
157/157 [==============================] - 256s 2s/step - loss: 0.6980 - accuracy: 0.4977 - val_loss: 0.6955 - val_accuracy: 0.4946
Epoch 1/4
157/157 [==============================] - 299s 2s/step - loss: 0.7393 - accuracy: 0.4949 - val_loss: 0.7082 - val_accuracy: 0.4946
Epoch 2/4
157/157 [==============================] - 286s 2s/step - loss: 0.7071 - accuracy: 0.5017 - val_loss: 0.7385 - val_accuracy: 0.5054
Epoch 3/4
157/157 [==============================] - 282s 2s/step - loss: 0.6973 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5054
Epoch 4/4
157/157 [==============================] - 295s 2s/step - loss: 0.6991 - accuracy: 0.5022 - val_loss: 0.6931 - v

KeyboardInterrupt: 

# Change the bounds

In [202]:
dimension = 5
total_birds = 10
max_iterations = 5
ssa = SSA_new(dimension=dimension, total_birds=total_birds, maximum_iterations=max_iterations)

lower_bound = [0.00001, 3, 10, 10, 50]
upper_bound = [0.01, 10, 50, 50, 100]

population = ssa.initialize_populations(lower_bound=lower_bound, upper_bound=upper_bound)

mse_vals_trial = []
min_mse_index = []
population_trials = []

for i in range(max_iterations):
    print("===================== Iteration " + str(i) + " =========================")
    mse_vals = train_model(population)
    mse_vals_trial.append(mse_vals)
    population_trials.append(population)
    print("Minimum mse = " + str(min(mse_vals)))
    min_index = mse_vals.index(min(mse_vals))
    min_mse_index.append(min_index)
    print("Populations corresponding to min mse = " + str(population[min_index]))
    population = get_new_population(ssa, population, mse_vals, lower_bound=lower_bound, upper_bound=upper_bound)

Inititializing parameters
===================== Iteration 0 =========================
Epoch 1/6
250/250 [==============================] - 307s 1s/step - loss: 0.7200 - accuracy: 0.4961 - val_loss: 0.7003 - val_accuracy: 0.4963
Epoch 2/6
250/250 [==============================] - 289s 1s/step - loss: 0.6945 - accuracy: 0.5084 - val_loss: 0.6983 - val_accuracy: 0.5037
Epoch 3/6
250/250 [==============================] - 298s 1s/step - loss: 0.6966 - accuracy: 0.5005 - val_loss: 0.6986 - val_accuracy: 0.4963
Epoch 4/6
250/250 [==============================] - 312s 1s/step - loss: 0.6963 - accuracy: 0.5049 - val_loss: 0.6952 - val_accuracy: 0.4963
Epoch 5/6
250/250 [==============================] - 346s 1s/step - loss: 0.6967 - accuracy: 0.5035 - val_loss: 0.6976 - val_accuracy: 0.5037
Epoch 6/6
250/250 [==============================] - 355s 1s/step - loss: 0.6957 - accuracy: 0.5010 - val_loss: 0.6942 - val_accuracy: 0.5037
Epoch 1/7
250/250 [==============================] - 282s 1s/s

IndexError: invalid index to scalar variable.

In [205]:
mse_vals

[0.2506100644000625,
 0.0818599622532408,
 0.2506278428574637,
 0.25959280488673214,
 0.25052548088351645,
 0.2503549454063748,
 0.2509638179982047,
 0.10477642440087415,
 0.25325840186648113,
 0.12692405600532466]